# Análise de Métricas de Qualidade - 75QUA

Este notebook analisa as métricas CK extraídas de múltiplas releases de um projeto Java.

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import json

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## 1. Configuração

Configure o nome do projeto analisado:

In [5]:
# CONFIGURAÇÃO - Altere para o nome do seu projeto
PROJECT_NAME = "jsoup"
RESULTS_DIR = Path(f"/workspace/resultados/{PROJECT_NAME}")

print(f"Analisando projeto: {PROJECT_NAME}")
print(f"Diretório de resultados: {RESULTS_DIR}")

Analisando projeto: jsoup
Diretório de resultados: /workspace/resultados/jsoup


## 1.2 pyDriller

In [8]:
from pydriller import Repository
import pandas as pd
from pathlib import Path

# Caminho do repositório e onde salvar os resultados
repo_path = Path("/workspace/projetos/jsoup")
results_dir = Path("/workspace/resultados/jsoup")
results_dir.mkdir(parents=True, exist_ok=True)

# Extrair commits que correspondem a releases (tags ou mensagens típicas)
releases_data = []

for commit in Repository(str(repo_path)).traverse_commits():
    msg = commit.msg.lower()
    # Filtra mensagens com "release", "version" ou "tag"
    if any(k in msg for k in ["release", "version", "tag", "v1.", "v2.", "v3."]):
        releases_data.append({
            "hash": commit.hash,
            "author": commit.author.name,
            "date": commit.committer_date,
            "message": commit.msg.strip()
        })

# Salvar CSV de releases
releases_df = pd.DataFrame(releases_data)
releases_df.sort_values("date", inplace=True)
releases_path = results_dir / "releases.csv"
releases_df.to_csv(releases_path, index=False)

print(f"Arquivo salvo em: {releases_path}")
print(f"Total de releases detectadas: {len(releases_df)}")
releases_df.head(20)


Arquivo salvo em: /workspace/resultados/jsoup/releases.csv
Total de releases detectadas: 506


,hash,author,date,message
0,548ce13435a00bb447fbffdbca8c5ce3be752ee4,Jonathan Hedley,2011-07-02 21:11:39+10:00,Initial implementation of Tag.
1,97f29aa3541dac70b811f766dd58ba47ac34d58c,Jonathan Hedley,2011-07-02 21:11:39+10:00,"Ignore tag ""html"" in parse tree as created by doc"
2,9db51cf756951fa62a7670b70d314af2f0ffc197,Jonathan Hedley,2011-07-02 21:11:40+10:00,Implemented getElementsByTagName
3,4bd743a3a5492d1d185bd21f421e49ce3a27af7c,Jonathan Hedley,2011-07-02 21:11:41+10:00,"Support for data only tags (script, textarea etc)"
4,00734226f07e57a2d13f82a4b395b4bb7b9ee20b,Jonathan Hedley,2011-07-02 21:11:43+10:00,Output HTML correctly for <! ... > and <? ... ...
5,40979c984d6ceebd50dab1ba516babb1570841a7,Jonathan Hedley,2011-07-02 21:11:43+10:00,Don't register unknown tags.\n\nUnknown tags c...
6,e86fce8a0480ec35b76cd42c82b3466d724af082,Jonathan Hedley,2011-07-02 21:11:43+10:00,Tests parser for unknown tags.
7,83d1848144abf29b501d59e0c3ec41e40c9cf020,Jonathan Hedley,2011-07-02 21:11:45+10:00,Rename Element.addChild to appendChild.\n\nAls...
8,3205f0ef9409bb622f66315983830aa89d9313cf,Jonathan Hedley,2011-07-02 21:11:45+10:00,Removed StartTag
9,56a6c72c3e12e62c649f5155b7240bff993230ee,Jonathan Hedley,2011-07-02 21:11:46+10:00,Version is 0.1 until first beta release


## 2. Carregar Dados de Releases

In [10]:
# Carregar informações de releases
releases_df = pd.read_csv(RESULTS_DIR / "releases.csv")
releases_df['date'] = pd.to_datetime(releases_df['date'])
releases_df = releases_df.sort_values('date')

print(f"Total de releases: {len(releases_df)}")
print(f"Período: {releases_df['date'].min().date()} até {releases_df['date'].max().date()}")
releases_df.head(20)

Total de releases: 506
Período: 2011-07-02 até 2025-10-14


/tmp/ipykernel_29/1173780761.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  releases_df['date'] = pd.to_datetime(releases_df['date'])


,hash,author,date,message
0,548ce13435a00bb447fbffdbca8c5ce3be752ee4,Jonathan Hedley,2011-07-02 21:11:39+10:00,Initial implementation of Tag.
1,97f29aa3541dac70b811f766dd58ba47ac34d58c,Jonathan Hedley,2011-07-02 21:11:39+10:00,"Ignore tag ""html"" in parse tree as created by doc"
2,9db51cf756951fa62a7670b70d314af2f0ffc197,Jonathan Hedley,2011-07-02 21:11:40+10:00,Implemented getElementsByTagName
3,4bd743a3a5492d1d185bd21f421e49ce3a27af7c,Jonathan Hedley,2011-07-02 21:11:41+10:00,"Support for data only tags (script, textarea etc)"
4,00734226f07e57a2d13f82a4b395b4bb7b9ee20b,Jonathan Hedley,2011-07-02 21:11:43+10:00,Output HTML correctly for <! ... > and <? ... ...
5,40979c984d6ceebd50dab1ba516babb1570841a7,Jonathan Hedley,2011-07-02 21:11:43+10:00,Don't register unknown tags.\n\nUnknown tags c...
6,e86fce8a0480ec35b76cd42c82b3466d724af082,Jonathan Hedley,2011-07-02 21:11:43+10:00,Tests parser for unknown tags.
7,83d1848144abf29b501d59e0c3ec41e40c9cf020,Jonathan Hedley,2011-07-02 21:11:45+10:00,Rename Element.addChild to appendChild.\n\nAls...
8,3205f0ef9409bb622f66315983830aa89d9313cf,Jonathan Hedley,2011-07-02 21:11:45+10:00,Removed StartTag
9,56a6c72c3e12e62c649f5155b7240bff993230ee,Jonathan Hedley,2011-07-02 21:11:46+10:00,Version is 0.1 until first beta release


## 3. Carregar Métricas CK de Todas as Releases

In [13]:
# Consolidar métricas de todas as releases
all_metrics = []

for release_dir in sorted(RESULTS_DIR.glob('v*')):
    class_csv = release_dir / 'ck' / 'jsoupclass.csv'
    
    if class_csv.exists():
        df = pd.read_csv(class_csv)
        df['release'] = release_dir.name
        all_metrics.append(df)
        print(f"✓ {release_dir.name}: {len(df)} classes")
    else:
        print(f"✗ {release_dir.name}: metrics não encontradas")

if all_metrics:
    df_all = pd.concat(all_metrics, ignore_index=True)
    print(f"\nTotal de registros: {len(df_all)}")
else:
    print("ERRO: Nenhuma métrica encontrada!")

ERRO: Nenhuma métrica encontrada!


In [ ]:
# Visualizar estrutura dos dados
df_all.head()

## 4. Estatísticas Descritivas por Release

In [ ]:
# Agregar métricas por release
metrics_by_release = df_all.groupby('release').agg({
    'wmc': ['mean', 'median', 'std', 'max'],
    'dit': ['mean', 'median', 'std', 'max'],
    'noc': ['mean', 'median', 'std', 'max'],
    'cbo': ['mean', 'median', 'std', 'max'],
    'lcom': ['mean', 'median', 'std', 'max'],
    'rfc': ['mean', 'median', 'std', 'max'],
    'loc': ['sum', 'mean', 'median', 'std']
}).round(2)

metrics_by_release

## 5. Visualização da Evolução das Métricas

In [ ]:
# Gráfico de evolução - WMC (Weighted Methods per Class)
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Evolução das Métricas CK', fontsize=16, fontweight='bold')

# WMC
metrics_by_release[('wmc', 'mean')].plot(ax=axes[0, 0], marker='o', color='blue')
axes[0, 0].set_title('WMC (Weighted Methods per Class) - Média')
axes[0, 0].set_xlabel('Release')
axes[0, 0].set_ylabel('WMC Médio')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# CBO
metrics_by_release[('cbo', 'mean')].plot(ax=axes[0, 1], marker='s', color='green')
axes[0, 1].set_title('CBO (Coupling Between Objects) - Média')
axes[0, 1].set_xlabel('Release')
axes[0, 1].set_ylabel('CBO Médio')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# LCOM
metrics_by_release[('lcom', 'mean')].plot(ax=axes[1, 0], marker='^', color='red')
axes[1, 0].set_title('LCOM (Lack of Cohesion of Methods) - Média')
axes[1, 0].set_xlabel('Release')
axes[1, 0].set_ylabel('LCOM Médio')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# LOC
metrics_by_release[('loc', 'sum')].plot(ax=axes[1, 1], marker='D', color='purple')
axes[1, 1].set_title('LOC (Lines of Code) - Total')
axes[1, 1].set_xlabel('Release')
axes[1, 1].set_ylabel('LOC Total')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'metrics_evolution.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Distribuição das Métricas

In [ ]:
# Boxplots das métricas
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Distribuição das Métricas CK', fontsize=16, fontweight='bold')

metrics = ['wmc', 'dit', 'noc', 'cbo', 'lcom', 'rfc']
positions = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2)]

for metric, pos in zip(metrics, positions):
    df_all.boxplot(column=metric, by='release', ax=axes[pos], rot=45)
    axes[pos].set_title(f'{metric.upper()}')
    axes[pos].set_xlabel('')
    
plt.suptitle('Distribuição das Métricas CK por Release', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'metrics_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Correlação entre Métricas

In [ ]:
# Matriz de correlação
correlation_metrics = ['wmc', 'dit', 'noc', 'cbo', 'lcom', 'rfc', 'loc']
corr_matrix = df_all[correlation_metrics].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlação entre Métricas CK', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Top Classes com Problemas de Qualidade

In [ ]:
# Classes com maior complexidade (WMC) na última release
latest_release = df_all[df_all['release'] == df_all['release'].unique()[-1]]

print("Top 10 Classes com Maior Complexidade (WMC):")
print(latest_release.nlargest(10, 'wmc')[['class', 'wmc', 'cbo', 'lcom', 'loc']])

print("\nTop 10 Classes com Maior Acoplamento (CBO):")
print(latest_release.nlargest(10, 'cbo')[['class', 'wmc', 'cbo', 'lcom', 'loc']])

print("\nTop 10 Classes com Menor Coesão (LCOM):")
print(latest_release.nlargest(10, 'lcom')[['class', 'wmc', 'cbo', 'lcom', 'loc']])

## 9. Análise de Tendências

In [ ]:
# Calcular taxa de crescimento das métricas
first_release = metrics_by_release.iloc[0]
last_release = metrics_by_release.iloc[-1]

growth_rates = pd.DataFrame({
    'Métrica': ['WMC', 'DIT', 'NOC', 'CBO', 'LCOM', 'RFC', 'LOC (total)'],
    'Primeira Release': [
        first_release[('wmc', 'mean')],
        first_release[('dit', 'mean')],
        first_release[('noc', 'mean')],
        first_release[('cbo', 'mean')],
        first_release[('lcom', 'mean')],
        first_release[('rfc', 'mean')],
        first_release[('loc', 'sum')]
    ],
    'Última Release': [
        last_release[('wmc', 'mean')],
        last_release[('dit', 'mean')],
        last_release[('noc', 'mean')],
        last_release[('cbo', 'mean')],
        last_release[('lcom', 'mean')],
        last_release[('rfc', 'mean')],
        last_release[('loc', 'sum')]
    ]
})

growth_rates['Variação (%)'] = ((growth_rates['Última Release'] - growth_rates['Primeira Release']) / growth_rates['Primeira Release'] * 100).round(2)

print("Análise de Crescimento das Métricas:")
print(growth_rates)

## 10. Exportar Resultados Consolidados

In [ ]:
# Salvar métricas agregadas
metrics_by_release.to_csv(RESULTS_DIR / 'metrics_summary_statistical.csv')
print(f"Estatísticas salvas em: {RESULTS_DIR / 'metrics_summary_statistical.csv'}")

# Salvar taxa de crescimento
growth_rates.to_csv(RESULTS_DIR / 'growth_rates.csv', index=False)
print(f"Taxas de crescimento salvas em: {RESULTS_DIR / 'growth_rates.csv'}")

print("\nAnálise completa!")